### Semantic Kernel Agent using MCP Tools via Azure API Management

Model Context Protocol with Azure API Management to enable plug & play of tools for a Semantic Kernel Agent
![flow](../utils/media/model-context-protocol.gif)

0️⃣ Initialize Modules

In [1]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPSsePlugin
from dotenv import load_dotenv
import os

load_dotenv(override=True)

apim_resource_gateway_url=os.getenv("APIM_RESOURCE_GATEWAY_URL")
mcp_server_url="http://localhost:8080/agent"
apim_subscription_key=os.getenv("APIM_SUBSCRIPTION_KEY")
openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION")               
openai_deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

1️⃣ Create the Semantic Kernel Orchestrator

In [2]:
async def main(user_input: str):
    # 1. Create the agent
    async with MCPSsePlugin(
        name="AgentPlugin",
        url=f"{mcp_server_url}/sse",
        description="Menu Agent Plugin",
    ) as agent_plugin:
              
        orchestrator = ChatCompletionAgent(
            service=AzureChatCompletion(
                endpoint=apim_resource_gateway_url,
                api_key=apim_subscription_key,
                api_version=openai_api_version,                
                deployment_name=openai_deployment_name
            ),
            name="Orchestrator",
            instructions="Route the request to the appropriate agent.",
            plugins=[agent_plugin],
        )

        thread: ChatHistoryAgentThread | None = None

        print(f"# User: {user_input}")
        # 2. Invoke the agent for a response
        response = await orchestrator.get_response(messages=user_input, thread=thread)
        print(f"# {response.name}: {response} ")
        thread = response.thread # type: ignore

        # 3. Cleanup: Clear the thread
        await thread.delete() if thread else None


In [ ]:
user_input = "Specials from the menu"
await main(user_input)

Error in post_writer: Client error '404 Not Found' for url 'http://localhost:8080/messages/?session_id=0f4b2dfaecc6460fabe05f781745e2c4'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404


In [ ]:
import os, json, asyncio, time, requests
from mcp import ClientSession
from mcp.client.sse import sse_client
import nest_asyncio
nest_asyncio.apply()

async def list_tools(server_url, authorization_header = None):
    headers = {"Authorization": authorization_header} if authorization_header else None
    async with sse_client(server_url, headers) as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()

            response = await session.list_tools()
            tools = response.tools
    print(f"✅ Connected to server {server_url}")
    print("⚙️ Tools:")
    for tool in tools:
        print(f"  - {tool.name}")
        print(f"     Input Schema: {tool.inputSchema}")
    
asyncio.run(list_tools(f"{mcp_server_url}/sse"))

Error in post_writer: Client error '404 Not Found' for url 'http://localhost:8080/messages/?session_id=f1cf314173f545ada78d1ed8394f413a'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
